In [2]:
import os
import time
import logging
import threading
from pathlib import Path
from concurrent.futures import ThreadPoolExecutor, as_completed

import pandas as pd
import requests

## How the “Step 4” Threaded Script Works

1. **Configuration**  
   - Defines file paths (`projects_ci.csv`, `ci_configs/`, `build_runs.csv`), a log file, request timeouts, sleep interval between requests (`0.5 s`), thread pool size (`MAX_WORKERS=5`), and a list of five GitHub tokens.

2. **`process_repo(idx, row)`**  
   - **Token rotation**: selects `TOKENS[idx % len(TOKENS)]` so each thread uses a different PAT in round-robin.  
   - **Download workflows**: GETs `.github/workflows` contents, saves each `.yml`/`.yaml` into `ci_configs/owner--repo/`.  
   - **Fetch latest run**: queries `/actions/runs?per_page=1` on the default branch; if found, retrieves job list to count jobs, collect runner names, and compute duration in minutes.  
   - **Return value**: a dict with extended fields (`run_id`, `run_number`, `workflow_name`, `event`, `head_sha`, `status`, `created_at`, `duration_min`, etc.), or `None` if no runs.

3. **`main()` Orchestration**  
   - Reads `projects_ci.csv` into a DataFrame and writes the header row to `build_runs.csv`.  
   - Ensures the `ci_configs/` directory exists.  
   - Spawns a `ThreadPoolExecutor` with 5 workers, submitting one `process_repo` task per row.  
   - As each future completes, under a thread-safe lock it appends non-`None` records to `build_runs.csv`.  
   - Logs progress every 100 repos and records all successes and errors.

---

## Smart Rate-Limit Workaround

- **Multiple Tokens**: rotates across five PATs (`TOKENS[idx % 5]`) to multiply available quota.  
- **Throttling**: adds a small `time.sleep(0.5)` after each API call to avoid bursts.  
- **Moderate Concurrency**: uses only 5 threads to balance speed and rate-limit safety.  

Together, these measures prevent 403s from GitHub’s secondary rate-limit/abuse detection while still leveraging parallelism for efficiency.  

In [ ]:
CI_CSV       = Path("projects_ci.csv")
CONFIGS_DIR  = Path("ci_configs")
BUILD_CI     = Path("build_runs.csv")   
LOG_FILE     = Path("ci_step4_threaded.log")
# Replace with your personal GitHub tokens
TOKENS = [
    "ghp_**********",  # Token 1
    "ghp_**********",  # Token 2
    "ghp_**********",  # Token 3
    "ghp_**********",  # Token 4
    "ghp_**********"   # Token 5
]

REQUEST_TIMEOUT = 15    # seconds
SLEEP_AFTER_REQ  = 0.5  # between calls
MAX_WORKERS      = 5

In [4]:
def setup_logger():
    logger = logging.getLogger("ci_step4_threaded")
    logger.setLevel(logging.INFO)
    fmt = logging.Formatter("%(asctime)s %(levelname)s %(message)s")
    fh = logging.FileHandler(LOG_FILE, mode="w", encoding="utf-8")
    fh.setFormatter(fmt)
    logger.addHandler(fh)
    ch = logging.StreamHandler()
    ch.setFormatter(fmt)
    logger.addHandler(ch)
    return logger

In [5]:
logger = setup_logger()
csv_lock = threading.Lock()

In [6]:
def get_headers(token):
    return {
        "Authorization": f"token {token}",
        "Accept":        "application/vnd.github.v3+json"
    }

In [6]:
def process_repo(idx, row):
    """
    For a single repo:
      1) download workflow YAMLs
      2) fetch latest run metadata
    Returns a dict with extended run fields, or None.
    """
    full_name      = row["name"]
    owner, repo    = full_name.split("/", 1)
    default_branch = row.get("defaultBranch", "main")

    # choose token
    token   = TOKENS[idx % len(TOKENS)]
    session = requests.Session()
    session.headers.update({
        "Authorization": f"token {token}",
        "Accept":        "application/vnd.github.v3+json"
    })

    logger.info(f"[{idx+1}] {owner}/{repo} (token #{(idx%len(TOKENS))+1})")

    # 1) download YAMLs
    wf_api = f"https://api.github.com/repos/{owner}/{repo}/contents/.github/workflows"
    try:
        resp = session.get(wf_api, timeout=REQUEST_TIMEOUT)
        resp.raise_for_status()
        items = resp.json()
        dest  = CONFIGS_DIR / f"{owner}--{repo}"
        dest.mkdir(parents=True, exist_ok=True)

        for item in items:
            if item.get("type")=="file" and item["name"].lower().endswith((".yml",".yaml")):
                try:
                    content = session.get(item["download_url"], timeout=REQUEST_TIMEOUT).text
                    (dest/item["name"]).write_text(content, encoding="utf-8")
                    logger.info(f"    saved {item['name']}")
                except Exception as e:
                    logger.error(f"    failed write {item['name']}: {e}", exc_info=True)
    except requests.HTTPError as he:
        if resp.status_code==404:
            logger.info("    no workflows folder")
        else:
            logger.error(f"    HTTP error: {he}", exc_info=True)
    except Exception as e:
        logger.error(f"    error fetching workflows: {e}", exc_info=True)

    time.sleep(SLEEP_AFTER_REQ)

    # 2) fetch latest run
    runs_api = (
        f"https://api.github.com/repos/{owner}/{repo}"
        f"/actions/runs?branch={default_branch}&per_page=1"
    )
    try:
        r = session.get(runs_api, timeout=REQUEST_TIMEOUT)
        r.raise_for_status()
        runs = r.json().get("workflow_runs", [])
        if not runs:
            logger.info("    no runs found")
            return None

        run = runs[0]
        # job details for runner names & count
        jobs_r = session.get(run["jobs_url"], timeout=REQUEST_TIMEOUT)
        jobs_r.raise_for_status()
        jobs = jobs_r.json().get("jobs", [])

        runner_names = sorted({j.get("runner_name","") for j in jobs})
        job_count    = len(jobs)

        # compute duration
        started = pd.to_datetime(run["run_started_at"])
        updated = pd.to_datetime(run["updated_at"])
        duration = (updated - started).total_seconds()/60.0

        return {
            # repo info
            "owner":         owner,
            "repo":          repo,
            # workflow run info
            "run_id":        run.get("id"),
            "run_number":    run.get("run_number"),
            "workflow_id":   run.get("workflow_id"),
            "workflow_name": run.get("name"),
            "event":         run.get("event"),
            "head_branch":   run.get("head_branch"),
            "head_sha":      run.get("head_sha"),
            "status":        run.get("status"),
            "conclusion":    run.get("conclusion"),
            "created_at":    run.get("created_at"),
            "run_started_at":run.get("run_started_at"),
            "updated_at":    run.get("updated_at"),
            "html_url":      run.get("html_url"),
            # job-level stats
            "job_count":     job_count,
            "runner_names":  ",".join(runner_names),
            "duration_min":  duration
        }

    except Exception as e:
        logger.error(f"    error fetching run metadata: {e}", exc_info=True)
        return None
    finally:
        time.sleep(SLEEP_AFTER_REQ)

In [7]:
def main():
    logger.info("Starting threaded Step 4 with extended fields")
    if not CI_CSV.exists():
        logger.error(f"Missing input: {CI_CSV}")
        return

    df = pd.read_csv(CI_CSV)
    total = len(df)
    logger.info(f"{total} repos to process")

    CONFIGS_DIR.mkdir(exist_ok=True)
    # prepare HEADER
    header = [
        "owner","repo","run_id","run_number","workflow_id","workflow_name",
        "event","head_branch","head_sha","status","conclusion",
        "created_at","run_started_at","updated_at","html_url",
        "job_count","runner_names","duration_min"
    ]
    pd.DataFrame(columns=header).to_csv(BUILD_CI, index=False)

    found = 0
    with ThreadPoolExecutor(max_workers=MAX_WORKERS) as executor:
        futures = {executor.submit(process_repo, idx, row): idx
                   for idx, row in df.iterrows()}

        for future in as_completed(futures):
            idx = futures[future]
            try:
                rec = future.result()
                if rec:
                    with csv_lock:
                        pd.DataFrame([rec]).to_csv(BUILD_CI,
                                                   mode="a",
                                                   header=False,
                                                   index=False)
                    found += 1
            except Exception as e:
                logger.error(f"task idx={idx} exception: {e}", exc_info=True)

            # progress every 100
            if (idx+1) % 100 == 0 or (idx+1)==total:
                print(f"[{idx+1}/{total}] done, {found} runs recorded")

    logger.info(f"Finished: {found}/{total} run records written to {BUILD_CI}")
    print(f"Done Step 4: {found}/{total} saved to {BUILD_CI}")

if __name__=="__main__":
    main()

2025-06-26 21:24:12,313 INFO Starting threaded Step 4 with extended fields
2025-06-26 21:24:12,557 INFO 14763 repos to process
2025-06-26 21:24:12,570 INFO [1] apache/cordova-android (token #1)
2025-06-26 21:24:12,571 INFO [2] snowballstem/snowball-website (token #2)
2025-06-26 21:24:12,571 INFO [3] dgarijo/widoco (token #3)
2025-06-26 21:24:12,571 INFO [4] sumeetchhetri/gatf (token #4)
2025-06-26 21:24:12,572 INFO [5] vert-x3/vertx-unit (token #5)
2025-06-26 21:24:13,858 INFO     saved ci.yml
2025-06-26 21:24:13,937 INFO     saved ci-build.yml
2025-06-26 21:24:13,984 INFO     saved update-website.yml
2025-06-26 21:24:14,006 INFO     saved ci.yml
2025-06-26 21:24:14,147 INFO     saved ci-4.x.yml
2025-06-26 21:24:14,296 INFO     saved release-audit.yml
2025-06-26 21:24:14,356 INFO     saved ci-release.yml
2025-06-26 21:24:14,688 INFO     saved ci-5.x-stable.yml
2025-06-26 21:24:14,838 INFO     saved codeql-analysis.yml
2025-06-26 21:24:15,226 INFO     saved ci-5.x.yml
2025-06-26 21:24:1

[100/14763] done, 82 runs recorded


2025-06-26 21:25:28,721 INFO [105] naugtur/handsfreeyoutube (token #5)
2025-06-26 21:25:28,733 INFO     no runs found
2025-06-26 21:25:28,995 INFO     saved codeql-analysis.yml
2025-06-26 21:25:29,242 INFO [106] orgapp/orgajs (token #1)
2025-06-26 21:25:29,413 INFO     saved codeql-analysis.yml
2025-06-26 21:25:29,694 INFO     saved nodejs.yml
2025-06-26 21:25:29,820 INFO [107] djdeveloperr/deno-canvas (token #2)
2025-06-26 21:25:29,853 INFO     saved main.yml
2025-06-26 21:25:30,333 INFO     saved ci.yml
2025-06-26 21:25:30,392 INFO     saved pr-build.yml
2025-06-26 21:25:30,823 INFO     saved release.yml
2025-06-26 21:25:30,853 INFO     saved soak-testing.yml
2025-06-26 21:25:30,854 INFO     saved publish.yml
2025-06-26 21:25:31,003 INFO [108] josa42/coc-sh (token #3)
2025-06-26 21:25:31,324 INFO     saved website.yml
2025-06-26 21:25:31,344 INFO     saved stale-bot.yml
2025-06-26 21:25:31,485 INFO [109] jermbo/sampleapis (token #4)
2025-06-26 21:25:31,763 INFO     no runs found
2025

[200/14763] done, 172 runs recorded


2025-06-26 21:26:39,442 INFO     saved static.yml
2025-06-26 21:26:39,722 INFO     saved ci.yml
2025-06-26 21:26:39,759 INFO [207] textlint-rule/textlint-rule-write-good (token #2)
2025-06-26 21:26:39,976 INFO [208] foretagsplatsen/klassified (token #3)
2025-06-26 21:26:39,988 INFO [209] rexxars/react-lowlight (token #4)
2025-06-26 21:26:40,331 INFO     no runs found
2025-06-26 21:26:40,791 INFO     saved test.yml
2025-06-26 21:26:40,838 INFO [210] ivanzusko/mostly-adequate-guide-uk (token #5)
2025-06-26 21:26:41,096 INFO     saved test.yml
2025-06-26 21:26:41,114 INFO     saved pages.yml
2025-06-26 21:26:41,591 INFO     saved tests.yml
2025-06-26 21:26:41,619 INFO [211] hubotio/hubot-diagnostics (token #1)
2025-06-26 21:26:41,962 INFO     saved build.yml
2025-06-26 21:26:42,651 INFO [212] jlobos/instagram-stories (token #2)
2025-06-26 21:26:42,721 INFO     saved pipeline.yml
2025-06-26 21:26:43,006 INFO [213] robinmalburn/redux-persist-chrome-storage (token #3)
2025-06-26 21:26:43,451

...

[14700/14763] done, 12624 runs recorded


2025-06-27 00:17:23,722 INFO [14706] style-dictionary/style-dictionary (token #1)
2025-06-27 00:17:23,823 INFO [14707] lustre-labs/lustre (token #2)
2025-06-27 00:17:23,985 INFO     saved codeql.yml
2025-06-27 00:17:23,997 INFO     saved npm-publish.yml
2025-06-27 00:17:24,759 INFO [14708] BryceWG/Blinko-Extention (token #3)
2025-06-27 00:17:24,768 INFO     saved release.yml
2025-06-27 00:17:24,854 INFO     no runs found
2025-06-27 00:17:24,908 INFO     no runs found
2025-06-27 00:17:24,908 INFO     saved pull_request.yml
2025-06-27 00:17:25,210 INFO     saved verify.yml
2025-06-27 00:17:25,318 INFO     saved release.yml
2025-06-27 00:17:25,357 INFO [14709] ExodusMovement/xpub-converter (token #4)
2025-06-27 00:17:25,414 INFO [14710] Kikobeats/null-prototype-object (token #5)
2025-06-27 00:17:25,730 INFO     saved release.yml
2025-06-27 00:17:26,298 INFO     no runs found
2025-06-27 00:17:26,399 INFO     saved main.yml
2025-06-27 00:17:26,439 INFO     saved codeql-analysis.yml
2025-06-

[14763/14763] done, 12674 runs recorded


2025-06-27 00:18:06,151 INFO Finished: 12675/14763 run records written to build_runs.csv


Done Step 4: 12675/14763 saved to build_runs.csv
